In [1]:
import os
import re

import warnings
warnings.simplefilter("ignore", UserWarning)
from matplotlib import pyplot as plt
%matplotlib inline


import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np 
from string import punctuation

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, auc, roc_auc_score
from sklearn.externals import joblib

import scipy
from scipy.sparse import hstack

In [14]:
import pandas as pd
import numpy as np
import xlrd
xls = pd.ExcelFile(r"C:\Users\ABHI\Desktop\acads\sem 3\capstone\project\Prospects 2018.xlsx")
summary = pd.read_excel(xls, 'Sheet1')
summary = summary.iloc[:,2:4]
summary = summary.replace(np.nan, 'n', regex=True)
summary.describe()
summary.columns = ['label', 'summary']
#print(summary)
summary_cleaned = pd.DataFrame(columns=summary.columns,index=summary.index)
print(summary_cleaned)



     label summary
0      NaN     NaN
1      NaN     NaN
2      NaN     NaN
3      NaN     NaN
4      NaN     NaN
5      NaN     NaN
6      NaN     NaN
7      NaN     NaN
8      NaN     NaN
9      NaN     NaN
10     NaN     NaN
11     NaN     NaN
12     NaN     NaN
13     NaN     NaN
14     NaN     NaN
15     NaN     NaN
16     NaN     NaN
17     NaN     NaN
18     NaN     NaN
19     NaN     NaN
20     NaN     NaN
21     NaN     NaN
22     NaN     NaN
23     NaN     NaN
24     NaN     NaN
25     NaN     NaN
26     NaN     NaN
27     NaN     NaN
28     NaN     NaN
29     NaN     NaN
...    ...     ...
1583   NaN     NaN
1584   NaN     NaN
1585   NaN     NaN
1586   NaN     NaN
1587   NaN     NaN
1588   NaN     NaN
1589   NaN     NaN
1590   NaN     NaN
1591   NaN     NaN
1592   NaN     NaN
1593   NaN     NaN
1594   NaN     NaN
1595   NaN     NaN
1596   NaN     NaN
1597   NaN     NaN
1598   NaN     NaN
1599   NaN     NaN
1600   NaN     NaN
1601   NaN     NaN
1602   NaN     NaN
1603   NaN  

In [10]:
# Importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import numpy as np
from collections import Counter
 
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
 
# Cleaning the text sentences so that punctuation marks, stop words & digits are removed
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    processed = re.sub(r"\d+","",normalized)
    y = processed.split()
    return y

In [20]:
train_clean_sentences = []
for i in range(len(summary_cleaned)):
    summary_cleaned.iloc[i,0] = summary.iloc[i,0]
    summary_cleaned.iloc[i,1] = (' '.join(clean(summary.iloc[i,1]))).strip()
    train_clean_sentences.append(summary_cleaned.iloc[i,1])
    
    
print(summary.head)
print('******')
print(summary_cleaned)


<bound method NDFrame.head of      label                                            summary
0        n  ​This Funding Opportunity Announcement (FOA), ...
1        n  ​Population-based Research to Optimize the Scr...
2        n  ​This Funding Opportunity Announcement (FOA) i...
3        y  ​\nThe purpose of this notice is to announce t...
4        n  ​This Funding Opportunity Announcement (FOA) i...
5        y  ​\nThe Washington Health Care Authority (HCA) ...
6        n  ​\nThe National Highway Traffic  Safety Admini...
7        y  ​The State of Oregon, acting by and through th...
8        n  ​The United States Coast Guard (USCG) Boating ...
9        n  ​\nThe Oakland Fund for Children and Youth (OF...
10       y  ​The Massachusetts Department of Early Educati...
11       n  ​An academic institution or goverment must be ...
12       y  ​The purpose of this Funding Opportunity Annou...
13       y  ​The Mayor’s Fund seeks an independent evaluat...
14       n  ​\nThis is a small buisness 

In [22]:
x_train, x_test, y_train, y_test = train_test_split(summary_cleaned['summary'], 
                                                    summary_cleaned['label'], 
                                                    test_size=0.1, 
                                                    random_state=42,
                                                    stratify=summary_cleaned['label'])

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1451,) (162,) (1451,) (162,)


In [24]:
vectorizer_word = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='word', 
                             stop_words='english', 
                             ngram_range=(1, 2))

vectorizer_word.fit(x_train)

tfidf_matrix_word_train = vectorizer_word.transform(x_train)
tfidf_matrix_word_test = vectorizer_word.transform(x_test)

In [25]:
lr_word = LogisticRegression(solver='sag', verbose=2)
lr_word.fit(tfidf_matrix_word_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 17 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='sag',
          tol=0.0001, verbose=2, warm_start=False)

In [26]:
y_pred_word = lr_word.predict(tfidf_matrix_word_test)
print(accuracy_score(y_test, y_pred_word))


0.9012345679012346


In [29]:
vectorizer_char = TfidfVectorizer(max_features=40000,
                             min_df=5, 
                             max_df=0.5, 
                             analyzer='char', 
                             ngram_range=(1, 4))

vectorizer_char.fit(x_train)

tfidf_matrix_char_train = vectorizer_char.transform(x_train)
tfidf_matrix_char_test = vectorizer_char.transform(x_test)

lr_char = LogisticRegression(solver='sag', verbose=2)
lr_char.fit(tfidf_matrix_char_train, y_train)

y_pred_char = lr_char.predict(tfidf_matrix_char_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 24 epochs took 2 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s finished


In [30]:
print(accuracy_score(y_test, y_pred_char))


0.9012345679012346


In [31]:
tfidf_matrix_word_char_train =  hstack((tfidf_matrix_word_train, tfidf_matrix_char_train))
tfidf_matrix_word_char_test =  hstack((tfidf_matrix_word_test, tfidf_matrix_char_test))

lr_word_char = LogisticRegression(solver='sag', verbose=2)
lr_word_char.fit(tfidf_matrix_word_char_train, y_train)

y_pred_word_char = lr_word_char.predict(tfidf_matrix_word_char_test)
print(accuracy_score(y_test, y_pred_word_char))


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 45 epochs took 2 seconds
0.9074074074074074


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s finished


In [33]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model
from keras.models import Sequential

from keras.layers import Input, Dense, Embedding, Conv1D, Conv2D, MaxPooling1D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D

from keras.callbacks import Callback
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [34]:
MAX_NB_WORDS = 80000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)

tokenizer.fit_on_texts(summary_cleaned['summary'])

In [36]:
print(x_train[15])
print(tokenizer.texts_to_sequences([x_train[15]]))


​ june family smoking prevention tobacco control act fsptca signed law act give authority food drug administration fda enact facilitate tobacco product regulation overall goal prevent americans—especially youth—from starting use tobacco encourage current user quit decrease harm tobacco product use act fda’s center tobacco product ctp created regulate cigarette cigarette tobacco rollyourown tobacco smokeless tobacco fda finalized deeming rule extending fda’s authority include regulation electronic nicotine delivery system such ecigarettes vape pen cigar hookah waterpipe tobacco pipe tobacco nicotine gel among others deeming rule went effect august overarching objective rfi obtain declaration technical capability various information data material qualified business concern specific project objective understand hookah commercial landscape includes competitive set manufacturing distribution hookah equipment smoked product hookah tobacco steam stone distribution channel subset endusers cust

In [37]:
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [38]:
MAX_LENGTH = 100
padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LENGTH)
padded_test_sequences = pad_sequences(test_sequences, maxlen=MAX_LENGTH)

In [40]:
padded_train_sequences
padded_train_sequences.shape

(1451, 100)

In [41]:
def get_simple_rnn_model():
    embedding_dim = 300
    embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))
    
    inp = Input(shape=(MAX_LENGTH, ))
    x = Embedding(input_dim=MAX_NB_WORDS, output_dim=embedding_dim, input_length=MAX_LENGTH, 
                  weights=[embedding_matrix], trainable=True)(inp)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(GRU(100, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(1, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

rnn_simple_model = get_simple_rnn_model()

In [46]:
import pydot
from keras.utils.vis_utils import plot_model

plot_model(rnn_simple_model, 
           to_file='rnn_simple_model.png', 
           show_shapes=True, 
           show_layer_names=True)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [72]:
print(type(y_train))
#y_train = pd.factorize(y_train)
#y_test = pd.factorize(y_test)
print (np.array(y_test[0]))
y_train = np.array(y_train[0])
y_test = np.array(y_test[0])


<class 'tuple'>
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [78]:
filepath="weights-improvement-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

batch_size = 256
epochs = 2

history = rnn_simple_model.fit(x=padded_train_sequences, 
                    y=y_train, 
                    validation_data=(padded_test_sequences, y_test), 
                    batch_size=batch_size, 
                    callbacks=[checkpoint], 
                    epochs=epochs, 
                    verbose=1)

best_rnn_simple_model = load_model('weights-improvement-01-0.9012.hdf5')

y_pred_rnn_simple = best_rnn_simple_model.predict(padded_test_sequences, verbose=1, batch_size=2048)

y_pred_rnn_simple = pd.DataFrame(y_pred_rnn_simple, columns=['prediction'])
y_pred_rnn_simple['prediction'] = y_pred_rnn_simple['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
y_pred_rnn_simple.to_csv('y_pred_rnn_simple.csv', index=False)

Train on 1451 samples, validate on 162 samples
Epoch 1/2
1451/1451 [==============================] - 25s 17ms/step - loss: 0.2973 - acc: 0.9035 - val_loss: 0.2963 - val_acc: 0.9012

Epoch 00001: val_acc improved from -inf to 0.90123, saving model to weights-improvement-01-0.9012.hdf5
Epoch 2/2
1451/1451 [==============================] - 24s 17ms/step - loss: 0.2904 - acc: 0.9035 - val_loss: 0.2986 - val_acc: 0.9012

Epoch 00002: val_acc did not improve from 0.90123
162/162 [==============================] - 1s 8ms/step


In [80]:
y_pred_rnn_simple = pd.read_csv('y_pred_rnn_simple.csv')
print(accuracy_score(y_test, y_pred_rnn_simple))

0.9012345679012346
